# Vectorizacion de texto

# MODELO 1.

In [36]:
# http://brandonrose.org/clustering

# Importo las librerias necesarias
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
#import mpld3

In [37]:
#  TOPIC MODELING
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import string
from nltk.corpus import stopwords
import json
import glob
import re

def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = pd.read_csv(f)
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)


In [63]:
def remove_stops(text, stops):
    text = re.sub(r'(I{1,3}|IV|V|VI{1,3}|IX|X)[-.) ]+','',text)
    text = re.sub(r'[^\w\s -]','',text)
    text = re.sub(r'-',' ',text)
    text = text.lower()
    words = text.split()
    final = []
    for word in words:
        if word not in stops:
            final.append(word)
    final = " ".join(final)
    final = final.translate(str.maketrans("", "", string.punctuation))
    final = "".join([i for i in final if not i.isdigit()])
    while "  " in final:
        final = final.replace("  ", " ")
    return (final)

def clean_docs(docs):
    stops = stopwords.words("spanish")
    final = []
    for doc in docs:
        clean_doc = remove_stops(doc, stops)
        final.append(clean_doc)
    return (final)

In [64]:
texto = load_data("titulosEncontrados.csv")

# print (descriptions[0])

cleaned_docs = clean_docs(texto['titulo'])
# print (cleaned_docs[0])


In [144]:
stops = stopwords.words("spanish")

vectorizer = TfidfVectorizer(
                                lowercase=True,
                                use_idf=True,
                                max_features=20000,
                                max_df=0.80,
                                min_df=100,
                                ngram_range = (1,3),
                                stop_words = stops
                                

                            )

vectors = vectorizer.fit_transform(cleaned_docs)

feature_names = vectorizer.get_feature_names()

dense = vectors.todense()
denselist = dense.tolist()

all_keywords = []

for description in denselist:
    x=0
    keywords = []
    for word in description:
        if word > 0:
            keywords.append(feature_names[x])
        x=x+1
    all_keywords.append(keywords)


In [ ]:
print (all_keywords)

In [146]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(vectors)

In [150]:
dist

array([[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       [ 1.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
         1.00000000e+00,  1.00000000e+00,  9.15097915e-01],
       [ 1.00000000e+00,  1.00000000e+00, -2.22044605e-16, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       ...,
       [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [ 1.00000000e+00,  9.15097915e-01,  1.00000000e+00, ...,
         1.00000000e+00,  1.00000000e+00, -2.22044605e-16]])

In [120]:
true_k = 20

model = KMeans(n_clusters=true_k, init="k-means++", max_iter=100, n_init=1)

model.fit(vectors)

order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

with open ("clusters_results.txt", "w", encoding="utf-8") as f:
    for i in range(true_k):
        f.write(f"Cluster {i}")
        f.write("\n")
        for ind in order_centroids[i, :10]:
            f.write (' %s' % terms[ind],)
            f.write("\n")
        f.write("\n")
        f.write("\n")

In [128]:
terms

['actor', 'actora', 'antecedentes', 'antecedentes autos', 'antecedentes autos interes', 'autos', 'autos interes', 'autos interes medico', 'clinica', 'conclusiones', 'conclusiones medico', 'conclusiones medico legales', 'consideraciones', 'consideraciones medico', 'consideraciones medico legales', 'contestacion', 'contestacion puntos', 'contestacion puntos pericia', 'datos', 'estudios', 'examen', 'hechos', 'historia', 'historia clinica', 'interes', 'interes medico', 'interes medico legal', 'legal', 'legales', 'medico', 'medico legal', 'medico legales', 'medicolegales', 'pericia', 'personales', 'petitorio', 'puntos', 'puntos pericia', 'solicitados']

In [ ]:
order_centroids

## CountVectorizer

In [141]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

text = load_data("titulosEncontrados.csv")['titulo']
coun_vect = CountVectorizer()
count_matrix = coun_vect.fit_transform(cleaned_docs)
count_array = count_matrix.toarray()
dfCountVec = pd.DataFrame(data=count_array,columns = coun_vect.get_feature_names())
dfCountVec

,aa,aaec,aar,ab,abdomen,abodarje,abor,abordaje,abuso,ac,...,visu,vital,vivencial,viviana,viviera,vo,walter,wiberg,wiuerg,zambrano
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5771,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5772,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5773,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5774,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Pretty printing has been turned OFF
